In [137]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#chemin_fichier_excel = 'C:\\Users\\milos\\OneDrive\\Bureau\\M2\\Gestion quant\\projet KALMAN\\Appl_data.xlsx'
#df = pd.read_excel(chemin_fichier_excel)
#colonnes_selectionnees = ['Date', 'price'] 
#df_selection = df[colonnes_selectionnees]
#prix_apple=df['price']
#Dynamic State-Space Stochastic Volatility Model
class DDSSM :
    def __init__(self,N,T,p,mu,kappa,theta,xi,rho,S0=100,V0=0.20):
        # paramètre
        self.mu = mu
        self.kappa = kappa
        self.theta=theta
        self.xi = xi
        self.rho = rho
        self.N=N
        self.T=T
        self.p=p
        # Discrétisation temporelle
        dt = T / N
        # Génération de mouvements browniens corrélés
        #np.random.seed(42)
        self.Bt = np.random.normal(0, np.sqrt(dt), self.N)
        self.Zt = self.rho * self.Bt + np.sqrt(1 - self.rho**2) * np.random.normal(0, np.sqrt(dt), self.N)
        # Initialisation des vecteurs de prix et de volatilité
        self.S = np.zeros(N)
        self.V = np.zeros(N)
        self.logV=np.zeros(N)
        self.S[0]=S0
        self.S[1]=S0
        self.V[0]=V0
        self.V[1]=V0
        self.logV[0]=np.log10(V0)
        self.logV[1]=np.log10(V0)
        # Simulation du modèle 
        for i in range(2, self.N):
            self.logV[i] = np.log10(self.V[i-1]) + dt/self.V[i-1]*(self.kappa * (self.theta - self.V[i - 1]) - 1/2*self.xi**2*self.V[i-1]**(p-1) - self.rho*self.xi*self.V[i-1]**(p-0.5) * (self.mu[i-1] - 1/2*self.V[i-1])) + self.rho*self.xi*self.V[i-1]**(p-3/2)*(np.log(self.S[i-1]) - np.log(self.S[i-2])) + self.xi*self.V[i-1]**(p-1)*np.sqrt(dt)*np.sqrt(1-self.rho)*self.Zt[i-1] 
            self.V[i] = 10**self.logV[i]
            lnS = np.log(self.S[i-1]) + (self.mu[i-1] -self.V[i]/2)*dt + np.sqrt(dt)*np.sqrt(self.V[i])*self.Bt[i]
            self.S[i] = np.exp(lnS)

    def simulate(self):
        return self.V, self.S

    def disp(self):
        df = pd.DataFrame([self.S,self.V,self.Bt,self.Zt],index=['St','Vt','Bt','Zt']).T
        print(df)
        return
    
# Paramètres du modèle
N=252#len(prix_apple)
mu = [0.07 for i in range(N)]
kappa = 3
theta = 0.20
xi = 0.1
rho = -0.5
T=1
p=1/2 # Modèle de Heston
# Modèle 
H = DDSSM(N,T,p,mu,kappa,theta,xi,rho)
H.disp()
#print(mu)


             St        Vt        Bt        Zt
0    100.000000  0.200000  0.090062  0.014061
1    100.000000  0.200000  0.035024 -0.002163
2    100.055565  0.199867  0.023942 -0.039486
3    100.058840  0.199378  0.005352 -0.047131
4    100.056911  0.198902  0.003474 -0.034654
..          ...       ...       ...       ...
247   97.205956  0.194564 -0.091722  0.090596
248   97.122739  0.195616 -0.026779  0.058766
249   97.038025  0.196181 -0.027280  0.077756
250   97.125433  0.196883  0.036249 -0.128310
251   97.112096  0.195741 -0.000959  0.016628

[252 rows x 4 columns]
